In [1]:
!pip install langchain==0.3.7 langchain-core==0.3.15 langchain-google-genai==2.0.4 google-generativeai==0.8.3 protobuf==5.27.0

Weather Agent


In [9]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata


# Get API keys from environment variables

gemini_api_key = userdata.get("DAY1")
if not gemini_api_key:
    print("Warning: GEMINI_API_KEY not found. Gemini model will not run.")

import os, requests

GEOCODING_API_URL = "https://geocoding-api.open-meteo.com/v1/search"
FORECAST_URL = "https://api.open-meteo.com/v1/forecast"

def get_weather(city: str, country: str | None = None) -> dict:
    # 1) geocode
    params = {"name": city, "count": 1}
    r = requests.get(GEOCODING_API_URL, params=params, timeout=30)
    r.raise_for_status()
    data = r.json()
    if not data.get("results"):
        return {"error": f"could not find location for {city}"}
    lat = data["results"][0]["latitude"]
    lon = data["results"][0]["longitude"]


    # 2) weather
    w = requests.get(
        FORECAST_URL,
        params={"latitude": lat, "longitude": lon, "current_weather": True},
        timeout=30,
    )
    w.raise_for_status()
    return {
        "city": city,
        "coords": {"lat": lat, "lon": lon},
        "current_weather": w.json().get("current_weather"),
    }


get_weather("tokyo")

{'city': 'tokyo',
 'coords': {'lat': 35.6895, 'lon': 139.69171},
 'current_weather': {'time': '2025-12-03T10:45',
  'interval': 900,
  'temperature': 9.6,
  'windspeed': 7.6,
  'winddirection': 329,
  'is_day': 0,
  'weathercode': 0}}

In [10]:
# Define a simple chain template for easy switching between models
def create_roles_chain(model):
    prompt = ChatPromptTemplate.from_template("{input}")
    return prompt | model | StrOutputParser()

In [16]:
def gpt4o_mini_agent(user_input):
    # First, let GPT decide what to do
    # This agent will check the intent of the user
    # This agent will extract city name from the user query
    query_prompt = f"""You are an intelligent assistant with access to a weather API.

    If the user asks anything about the weather in a city, respond with:
    "CALL_WEATHER_API: <city-name>"

    Otherwise, answer normally.

    User input: "{user_input}"
    """

    print(query_prompt)

    gemini_model = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-lite-001",
        google_api_key=gemini_api_key,
        temperature=0.3 )

    print("---   Response ---")

    assistant_reply = create_roles_chain(gemini_model).invoke({"input": query_prompt})
    print(f"[Assistant Thought] {assistant_reply}")

    # If GPT indicates to call the API
    if assistant_reply.startswith("CALL_WEATHER_API:"):
        city_name = assistant_reply.replace("CALL_WEATHER_API:", "").strip()

        # Step 1: Get latitude and longitude
        api_result = get_weather(city_name)

        # Step 3: Let Model (GPT/Gemini) format the final answer nicely
        final_prompt = f"The weather information for {city_name} is: {api_result}. Please compose a friendly response to the user."

        gemini_model = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-lite-001",
        google_api_key=gemini_api_key,
        temperature=0.3 )

        print("---   Response [Final]---")
        final_answer = create_roles_chain(gemini_model).invoke({"input": final_prompt})

        return final_answer
    else:
        return assistant_reply

In [19]:
print(gpt4o_mini_agent("What is the weather in TamilNadu"))



You are an intelligent assistant with access to a weather API.

    If the user asks anything about the weather in a city, respond with:
    "CALL_WEATHER_API: <city-name>"

    Otherwise, answer normally.

    User input: "What is the weather in TamilNadu"
    
---   Response ---
[Assistant Thought] CALL_WEATHER_API: TamilNadu

---   Response [Final]---


"Oh no! It seems I'm having a little trouble finding the weather information for Tamil Nadu right now. I'm still learning and sometimes I get things mixed up. I'll keep trying to get that information for you, and hopefully, I'll have it soon! In the meantime, you might try searching for the weather in a specific city within Tamil Nadu, like Chennai or Madurai.  Thanks for your patience!"



In [20]:
import requests

EXCHANGE_API_URL = "https://open.er-api.com/v6/latest/"

def call_exchange_api(base: str, target: str) -> dict:
    r = requests.get(EXCHANGE_API_URL + base.upper(), timeout=15)
    if r.status_code != 200:
        return {"error": "could not retrieve exchange rates"}
    data = r.json()
    if data.get("result") != "success":
        return {"error": "exchange service returned failure"}
    rate = data["rates"].get(target.upper())
    if not rate:
        return {"error": f"target {target} not found"}
    return {
        "base": base.upper(),
        "target": target.upper(),
        "rate": rate,
        "formatted": f"1 {base.upper()} = {rate} {target.upper()}",
    }

In [21]:
def gpt4o_mini_currency_agent(user_input):
    query_prompt = f"""You are an assistant with access to a currency exchange API.

    If the user asks about exchanging currency, respond exactly:
    "CALL_EXCHANGE_API: <base-currency-code> to <target-currency-code>"

    Otherwise, answer normally.

    User input: "{user_input}"
    """

    gemini_model = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-lite-001",
        google_api_key=gemini_api_key,
        temperature=0.3 )

    print("---   Response ---")

    assistant_reply = create_roles_chain(gemini_model).invoke({"input": query_prompt})
    print(f"[Assistant Thought] {assistant_reply}")


    if assistant_reply.startswith("CALL_EXCHANGE_API:"):
        parts = assistant_reply.replace("CALL_EXCHANGE_API:", "").strip().split(" to ")
        if len(parts) == 2:
            base_currency = parts[0]
            target_currency = parts[1]
            api_result = call_exchange_api(base_currency, target_currency)
            final_prompt = f"The exchange rate is: {api_result}. Please compose a friendly detailed response."
        else:
            return "Sorry, I couldn't understand the currency pair."
    else:
        return assistant_reply

    gemini_model = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-lite-001",
        google_api_key=gemini_api_key,
        temperature=0.3 )

    print("---   Response [Final]---")
    final_response = create_roles_chain(gemini_model).invoke({"input": final_prompt})

    return final_response

In [22]:
user_question = "How much is USD to INR?"
answer = gpt4o_mini_currency_agent(user_question)
print(answer)

---   Response ---
[Assistant Thought] CALL_EXCHANGE_API: USD to INR

---   Response [Final]---
Okay, I've got the exchange rate information! Let's break it down:

You've provided me with the following information about the exchange rate between the US Dollar (USD) and the Indian Rupee (INR):

*   **`base`: 'USD'** This tells us the currency we're starting with. In this case, it's the US Dollar.
*   **`target`: 'INR'** This tells us the currency we're converting to. In this case, it's the Indian Rupee.
*   **`rate`: 89.941202** This is the crucial number! It means that **1 US Dollar (USD) is currently worth 89.941202 Indian Rupees (INR)**.
*   **`formatted`: '1 USD = 89.941202 INR'** This is just a user-friendly way of presenting the information from the `rate` key. It's the same information, just written out in a more readable format.

**In simpler terms:**

If you have 1 US Dollar, you can exchange it for approximately 89.94 Indian Rupees.

**Important Considerations:**

*   **Exchan

Master Agent

In [23]:
import os, json

def route_intent(user_text: str) -> dict:

    query_prompt = f"""
          You are a tool router. You decide which external API to call.

          Available tools:
          - "weather.lookup" → args: {{"city":"string","country":"string|null"}}
          - "fx.convert" → args: {{"base":"string","target":"string"}}

          Rules:
          - If user asks about weather or temperature → "weather.lookup"
          - If user asks about exchange rate, currency, USD to INR → "fx.convert"
          - Otherwise return {{"tool_name":"none","args":{{}}}}

          Return ONLY valid JSON:
          {{"tool_name":"...", "args":{{...}}}}

          User: "{user_text}"
          """

    print(query_prompt)

    gemini_model = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-lite-001",
        google_api_key=gemini_api_key,
        temperature=0.3 )

    print("---   Response ---")

    assistant_reply = create_roles_chain(gemini_model).invoke({"input": query_prompt})
    print(f"[Assistant Thought] {assistant_reply}")
    cleaned = assistant_reply.strip()

    if cleaned.startswith("```"):
        cleaned = cleaned.split("```")[1]  # get inside the fence
    cleaned = cleaned.replace("json", "").strip()

    # ------------------------------------------------------------------
    # 2. Convert JSON string → Python dict
    # ------------------------------------------------------------------
    try:
        parsed = json.loads(cleaned)
    except Exception as e:
        print("JSON parse error:", e)
        parsed = {"tool_name": "none", "args": {}}

    return parsed

In [24]:
def finalize_answer(user_text: str, tool_result: dict) -> str:

    prompt = f"""
      You are a friendly assistant.
      User asked: "{ user_text }"
      Tool result (factual): "{ tool_result }"

      Write a short, friendly answer for the user.
      If tool_result contains an error, explain it briefly.
    """
    gemini_model = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-lite-001",
        google_api_key=gemini_api_key,
        temperature=0.3 )

    print("---   Response ---")
    reply = create_roles_chain(gemini_model).invoke({"input": prompt})
    print(f"[Assistant Thought] {reply}")

    return reply


In [25]:

def execute_tool(tool_name: str, args: dict) -> dict:
    if tool_name == "weather.lookup":
        return get_weather(args.get("city", ""), args.get("country"))
    if tool_name == "fx.convert":
        return call_exchange_api(args.get("base", "USD"), args.get("target", "INR"))
    if tool_name == "none":
        return {"info": "no tool needed"}
    return {"error": f"unknown tool {tool_name}"}


In [26]:

def master_agent(user_text: str) -> str:
    route = route_intent(user_text)
    print(route)
    tool_name = route.get("tool_name", "none")
    print(tool_name)
    args = route.get("args", {})
    tool_result = execute_tool(tool_name, args)
    print(tool_result)
    final_answer = finalize_answer(user_text, tool_result)
    return final_answer

In [31]:
#print(master_agent("what is the weather like in new delhi"))
print(master_agent("Why is the sky blue?"))


          You are a tool router. You decide which external API to call.

          Available tools:
          - "weather.lookup" → args: {"city":"string","country":"string|null"}
          - "fx.convert" → args: {"base":"string","target":"string"}

          Rules:
          - If user asks about weather or temperature → "weather.lookup"
          - If user asks about exchange rate, currency, USD to INR → "fx.convert"
          - Otherwise return {"tool_name":"none","args":{}}

          Return ONLY valid JSON:
          {"tool_name":"...", "args":{...}}

          User: "Why is the sky blue?"
          
---   Response ---


ResourceExhausted: 429 Resource exhausted. Please try again later. Please refer to https://cloud.google.com/vertex-ai/generative-ai/docs/error-code-429 for more details.